In [1]:
import sys
sys.path.insert(1, '../src/')
sys.path.insert(1, '../src/pddl-parser')

from pddl_parser.PDDL import PDDL_Parser

In [2]:
import json
import os
import numpy as np

In [3]:
class Planner:

    # -----------------------------------------------
    # Solve
    # -----------------------------------------------

    def solve(self, parser):
        # Parser
        # parser = PDDL_Parser()
        # parser.parse_domain(domain)
        # parser.parse_problem(problem)
        # Parsed data
        state = parser.state
        goal_pos = parser.positive_goals
        goal_not = parser.negative_goals
        # Do nothing
        if self.applicable(state, goal_pos, goal_not):
            return []
        # Grounding process
        ground_actions = []
        for action in parser.actions:
            for act in action.groundify(parser.objects, parser.types):
                ground_actions.append(act)
        # Search
        visited = set([state])
        fringe = [state, None]
        while fringe:
            state = fringe.pop(0)
            plan = fringe.pop(0)
            for act in ground_actions:
                if self.applicable(state, act.positive_preconditions, act.negative_preconditions):
                    new_state = self.apply(state, act.add_effects, act.del_effects)
                    if new_state not in visited:
                        if self.applicable(new_state, goal_pos, goal_not):
                            full_plan = [act]
                            while plan:
                                act, plan = plan
                                full_plan.insert(0, act)
                            return full_plan
                        visited.add(new_state)
                        fringe.append(new_state)
                        fringe.append((act, plan))
        return None

    # -----------------------------------------------
    # Applicable
    # -----------------------------------------------

    def applicable(self, state, positive, negative):
        return positive.issubset(state) and negative.isdisjoint(state)

    # -----------------------------------------------
    # Apply
    # -----------------------------------------------

    def apply(self, state, positive, negative):
        return state.difference(negative).union(positive)

# 1 Action Generation

In [4]:
# cache_dir = '../data/evaluation/cache/'
# sample_dir = '../data/evaluation/actions_generation/sample_output/'
# sample_metadata = json.load(open('../data/evaluation/actions_generation/sample_output/meta_data.json'))
# domain_header_dir = '../data/evaluation/actions_generation/{test_case}/domain_header.pddl'

# sample_metadata

In [5]:
# tmp_filepath = sample_dir + '1.txt'
# domain_header = ''.join(open(domain_header_dir.format(test_case = 1)).readlines())

# actions = ''.join(open(tmp_filepath).readlines())
# tmp_domain = domain_header + actions
# tmp_domain

# tmp_domain_file = cache_dir + 'tmp_action_generation.pddl'
# with open(cache_dir + 'tmp_action_generation.pddl', 'w') as file:
#     file.write(tmp_domain)

In [8]:
import re

- Summary
    - Case 1
        - Intrinsic Eval: X pass, Y parsing_error, X actions_incomplete
        - Extrinsic Eval:
            - Problem Excution: success_rate = Z (out of X)
            - ...
- Raw
    - Case 1
        - Output 1
            - Intrinsic Eval: pass/parsing_error/actions_incomplete
            - Extrinsic Eval:
                - Problem Excution: 
                    - problem 1: solved/unsolved
                    - ...
                - ...

In [9]:
true_dir = '../data/evaluation/actions_generation/true/'
pred_dir = '../data/evaluation/actions_generation/pred/'
cache_dir = '../data/evaluation/cache/'
class Tester:
    def __init__(self, cache_dir = '../data/evaluation/cache/'):
        self.cache_dir = cache_dir
    
    def check_action_list(self, true_dir, test_case, output_actions):
        ground_truth_actions = ''.join(open('{true_dir}{test_case}/actions.txt'.format(
                                    true_dir = true_dir,
                                    test_case = test_case
                                )).readlines())
        all_action_texts = [
            ac[:-1].replace(':action', '').strip()
            for ac in re.findall(':action\s+.+;', ground_truth_actions)
        ]
        found = 0
        for ac in all_action_texts:
            if re.search(':action\s+{}'.format(ac), output_actions):
                found += 1
        return found, len(all_action_texts)

    def eval_summary_stat(self, case_results_raw):
        summary = {}

        summary['intrinsic'] = {
            'pass': 0,
            'parsing_error': 0,
            'action_incomplete': 0
        }   
        for output_action_file in case_results_raw:
            summary['intrinsic'][case_results_raw[output_action_file]['intrinsic']] += 1

        summary['extrinsic'] = {}
        for output_action_file in case_results_raw:
            if case_results_raw[output_action_file]['extrinsic'] is None:
                continue
            
            for problem in case_results_raw[output_action_file]['extrinsic']:
                if problem not in summary['extrinsic']:
                    summary['extrinsic'][problem] = {'solved': 0, 'unsolved': 0}
                if case_results_raw[output_action_file]['extrinsic'][problem] == 'solved':
                    summary['extrinsic'][problem]['solved'] += 1
                else:
                    summary['extrinsic'][problem]['unsolved'] += 1
        
        for problem in summary['extrinsic']:
            summary['extrinsic'][problem]['succ_rate'] = summary['extrinsic'][problem]['solved'] / sum(summary['extrinsic'][problem].values())

        return summary

    def eval_action_generation(self, true_dir, pred_dir):
        self.true_dir = true_dir
        self.pred_dir = pred_dir
        self.meta_data = json.load(open(pred_dir + 'meta_data.json'))
        
        eval_results = {}
        for test_case in self.meta_data:
            case_results_raw = {}

            domain_header_fp = '{true_dir}/{test_case}/domain_header.pddl'.format(
                true_dir = true_dir,
                test_case = test_case
            )
            domain_header = ''.join(open(domain_header_fp).readlines())
            
            for output_action_file in self.meta_data[test_case]:
                case_results_raw[output_action_file] = {
                    'intrinsic': 'pass',
                    'extrinsic': None
                }
                # 0. getting the domain file
                output_actions = ''.join(open('{pred_dir}/{action_file}'.format(
                    pred_dir = pred_dir,
                    action_file = output_action_file
                )).readlines())
                pred_domain = domain_header + output_actions
                
                # 1. Intrinsic check domain file
                tmp_domain_file = cache_dir + 'tmp_action_generation.pddl'
                with open(cache_dir + 'tmp_action_generation.pddl', 'w') as file:
                    file.write(pred_domain)
                # 1.1 check if parse-able
                parser = PDDL_Parser()
                try:
                    parser.parse_domain(tmp_domain_file)
                except:
                    case_results_raw[output_action_file]['intrinsic'] = 'parsing_error'
                    continue
                # 1.2 check if actions are all found
                action_found, action_total = self.check_action_list(true_dir, test_case, output_actions)
                if action_total > action_found:
                    case_results_raw[output_action_file]['intrinsic'] = 'action_incomplete'
                    continue
                
                # 2. Extrinsic evaluations
                case_results_raw[output_action_file]['extrinsic'] = {}
                problem_dir = '{true_dir}/{test_case}/problems/'.format(
                    true_dir = true_dir,
                    test_case = test_case
                )
                for problem in os.listdir(problem_dir):
                    if '.pddl' not in problem:
                        continue
                    problem_file = '{true_dir}/{test_case}/problems/{problem}'.format(
                        true_dir = true_dir,
                        test_case = test_case,
                        problem = problem
                    )

                    plan = self.eval_unit_action_generation(tmp_domain_file, problem_file)
                    if plan:
                            case_results_raw[output_action_file]['extrinsic'][problem] = 'solved'
                    else:
                        case_results_raw[output_action_file]['extrinsic'][problem] = 'unsolved'
            
            summary = self.eval_summary_stat(case_results_raw)
            eval_results[test_case] = {
                'summary': summary,
                'raw': case_results_raw
            }
            
        return eval_results


    def eval_unit_action_generation(self, domain_file, problem_file):
        parser = PDDL_Parser()
        parser.parse_domain(domain_file)
        parser.parse_problem(problem_file)
        planner = Planner()
        plan = planner.solve(parser)
        return plan
        
T = Tester(cache_dir)

In [13]:
T.eval_action_generation(true_dir, pred_dir)['1']

{'summary': {'intrinsic': {'pass': 2,
   'parsing_error': 2,
   'action_incomplete': 3},
  'extrinsic': {'problem_1.pddl': {'solved': 1,
    'unsolved': 1,
    'succ_rate': 0.5},
   'problem_2.pddl': {'solved': 1, 'unsolved': 1, 'succ_rate': 0.5},
   'problem_3.pddl': {'solved': 1, 'unsolved': 1, 'succ_rate': 0.5}}},
 'raw': {'1.txt': {'intrinsic': 'pass',
   'extrinsic': {'problem_1.pddl': 'solved',
    'problem_2.pddl': 'solved',
    'problem_3.pddl': 'solved'}},
  '2.txt': {'intrinsic': 'pass',
   'extrinsic': {'problem_1.pddl': 'unsolved',
    'problem_2.pddl': 'unsolved',
    'problem_3.pddl': 'unsolved'}},
  '3.txt': {'intrinsic': 'parsing_error', 'extrinsic': None},
  '4.txt': {'intrinsic': 'action_incomplete', 'extrinsic': None},
  '5.txt': {'intrinsic': 'parsing_error', 'extrinsic': None},
  '6.txt': {'intrinsic': 'action_incomplete', 'extrinsic': None},
  '7.txt': {'intrinsic': 'action_incomplete', 'extrinsic': None}}}

In [83]:
print('Name: Doe\nPreferences: n/a\nInterests: Cooking, trying new recipes\nFinancial situation: sufficient\nOccupation: investment banker\nHobbies: driving, diving, playing golf\nGender: Not specified\nLifestyle: very busy, fast-pace')

Name: Doe
Preferences: n/a
Interests: Cooking, trying new recipes
Financial situation: sufficient
Occupation: investment banker
Hobbies: driving, diving, playing golf
Gender: Not specified
Lifestyle: very busy, fast-pace
